In [1]:
import mwparserfromhell as mwp
from nltk.corpus import stopwords
import re
import nltk
import json
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Briggstone\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def return_tokens_wo_stopwords (s):
    punctuations = set(['.', ',', ';', ':', '?', '!', '#', '\\', '/', '"', '\'', '\'\'', '´´', '´', '``', '`', '(', ')'])
    stop_words = set(stopwords.words('english'))
    filter = punctuations & stop_words
    s = re.sub(r'([^\s\w]|_)+', ' ', s)
    tokens = s.split()
    remove = set(tokens) &  stop_words
    tokens = [x for x in tokens if x.lower() not in remove]
    return tokens

def return_links (tl):
    link_list = []
    for l in tl:
        link_list.append(str(l.title))
    return link_list

def handle_text_string (s):
    s = mwp.parse(s)
    links = s.filter_wikilinks()
    links = return_links(links)
    s = s.strip_code()
    s = return_tokens_wo_stopwords(s)
    return s,links

In [3]:
%%time
articles_id = []
articles_info = []

with open('test14.xml-p7697599p7744799', encoding = "utf_8") as fp:
    in_article = False
    article_id = ''
    title = ""
    in_text = False
    text = []
    links = []
    for line in fp:
        line = line.strip()
        if in_article:
            if in_text:
                if line.find('/text') != -1:
                    in_text = False
                    s = line[0:line.find('<')]
                    s,l = handle_text_string(s)
                else:
                    s,l = handle_text_string(line)
                text.extend(s)
                links.extend(l)
            elif line.find('</page>') != -1 and in_article:
                in_article = False
                articles_id.append(article_id)
                article_info = []
                article_info.append(title)
                article_info.append(text)
                article_info.append(links)
                articles_info.append(article_info)
                title = ""
                text = []
                links = []
            elif line.find('<text') != -1:
                if line.find('#REDIRECT') != -1:
                        in_article = False
                        title = ""
                        id = ""
                else:
                    in_text = True               
            elif line.find('<title>') != -1 and not title:
                if line.find('Wikipedia:') != -1:
                    in_article = False
                else:
                    i = line.find('>') + 1
                    title = line[i: line.find('<', i)]
            elif line.find('<id>') != -1 and not article_id:
                i = line.find('>') + 1
                article_id = line[i: line.find('<', i)]
        elif line.find('<page>') != -1:
            in_article = True
        

Wall time: 12 s


In [15]:
print(articles_id[0])
print(articles_info[0])


7697605
['Konica Minolta Cup', ['Japan', 'LPGA', 'Championship', 'Konica', 'Minolta', 'Cup', 'golf', 'competition', 'WRU', 'Challenge', 'Cup', 'Welsh', 'rugby', 'union', 'competition', 'Konica', 'Cup', 'Minolta', 'merger', 'may', 'refer', 'Konica', 'Cup', 'football', 'football', 'competition'], ['Japan LPGA Championship', 'WRU Challenge Cup', 'Konica Cup (football)']]


In [17]:
j = json.dumps([{'article_id': article_id, 'article_info': article_info} for article_id, article_info in zip(articles_id, articles_info)])


In [20]:
print(j[0:1000])

[{"article_id": "7697605", "article_info": ["Konica Minolta Cup", ["Japan", "LPGA", "Championship", "Konica", "Minolta", "Cup", "golf", "competition", "WRU", "Challenge", "Cup", "Welsh", "rugby", "union", "competition", "Konica", "Cup", "Minolta", "merger", "may", "refer", "Konica", "Cup", "football", "football", "competition"], ["Japan LPGA Championship", "WRU Challenge Cup", "Konica Cup (football)"]]}, {"article_id": "7697605", "article_info": ["Archer (typeface)", ["Infobox", "typeface", "name", "Archer", "image", "ArcherSpec", "svg", "style", "Serif", "classifications", "Humanist", "slab", "serif", "date", "2001", "creator", "Tobias", "Frere", "Jones", "br", "Jonathan", "Hoefler", "foundry", "Hoefler", "Frere", "Jones", "Archer", "slab", "serif", "typeface", "designed", "2001", "Tobias", "Frere", "Jones", "Jonathan", "Hoefler", "use", "Martha", "Stewart", "Living", "magazine", "It", "later", "released", "Hoefler", "Frere", "Jones", "commercial", "licensing", "Structure", "The", "ty